# Crawler
 
lets call crawler and fetch some documents

In [1]:
!root=~/PycharmProjects/

In [2]:
!cd Crawler

In [3]:
from Crawler.crawler import Crawler

crawler = Crawler(downloaded=False)
print(crawler.downloaded)
crawler.get_collection()


False
file reut2-000 docs are being processed and extracted to ./.misc/docs/reut2-000


In [4]:
crawler.retrieve_docs() # retrieve random doc from the docs collection :3 

('./.misc/docs/reut2-000/858.txt',
 '\n 2-MAR-1987 18:34:32.90\n\nusa\n\n\n\n\n \n\x05\x05\x05F\n\x16\x16\x01f0323\x1freute\nr f BC-REUTERS-<RTRSY>-IN-RE   03-02 0106\n\x02\nREUTERS <RTRSY> IN REAL ESTATE MARKET VENTURE\n    NEW YORK, March 2 - Reuters Holdings PLC said its Reuters\nInformation Services Inc unit will join <Real Estate Financing\nPartnership>, Philadelphia, to offer an electronic market\naccess system for commercial property financing.\n    Reuters said the system, named Real Estate Select View\nProgram, or RSVP, will use its private communications network\nto provide a confidential method for the purchasing, selling\nand financing of commercial property.\n    The system, set for testing in August in selected U.S.\ncities, is expected to be operational 90 days after initial\ntesting, Reuters said.\n Reuter\n\x03\n')

In [5]:
from Indexer.indexer import Indexer
indexer = Indexer()

In [20]:
indexer(crawler.retrieve_docs())

In [23]:
indexer.search("in*y")

Query expression is ::   ($industry$) 


['./.misc/docs/reut2-000/657.txt']

In [24]:
max([len(indexer.aux_index[key]) for key in indexer.aux_index.keys()])

7